In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [1]:
from pickle import FALSE
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

from typing import Dict
from time import sleep

from tqdm import tqdm

def style_string_to_dict(style: str) -> Dict[str, str]:
  attributes = style.strip(";").split("; ")
  style_dict = {}
  for line in attributes:
    [key, value] = line.split(": ")
    style_dict[key] = value
  return style_dict

def get_jumbo_photo_from_style_string(style: str) -> str:
    style_dict = style_string_to_dict(style)
    return style_dict["background-image"].split('("')[1].split("?")[0]

def get_html_soup(url):
    print("Doing request!")
    driver.get(url)
    wait = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "zoomed-image"))
    )
    datos = driver.execute_script("return document.body.innerHTML")
    print("Request executed!")
    print("Request done!")
    return BeautifulSoup(datos, "lxml")

def get_codes(html_soup, info, tags, classes):
    found = False
    for tag in tags:
        for className in classes:
            code = (html_soup.find(tag, class_=className))
            if code:
                found = True
                break
        if found:
            break
    if found:
        try:
            code = code.string.split(" ")[1]
            info.append(code)
        except:
            info.append(None)
    else:
        info.append(None)
    return info

def get_names(html_soup, info, tags, classes):
    found = False
    for tag in tags:
        for className in classes:
            name = (html_soup.find(tag, class_=className))
            if name:
                found = True
                break
        if found:
            break
    if found:
        try:
            name = name.string
            info.append(name)
        except:
            info.append(None)
    else:
        info.append(None)
    return info

def get_photo(html_soup, info, tags, classes):
    found = False
    for tag in tags:
        for className in classes:
            style_photo = (html_soup.find(tag, class_=className))
            if style_photo:
                found = True
                break
        if found:
            break
    if found:
        try:
            photo = get_jumbo_photo_from_style_string(style_photo["style"])
            info.append(photo)
        except:
            info.append(None)
    else:
        info.append(None)
    return info

def get_price(html_soup, info, tags, classes):
    found = False
    for tag in tags:
        for className in classes:
            price = (html_soup.find(tag, class_=className))
            if price:
                found = True
                break
        if found:
            break
    if found:
        try:
            price = price.string.strip("$")
            info.append(price)
        except:
            info.append(None)
    else:
        info.append(None)
    return info

def get_price_kg(html_soup, info, tags, classes):
    found = False
    for tag in tags:
        for className in classes:
            price = (html_soup.find(tag, class_=className))
            if price:
                found = True
                break
        if found:
            break
    if found:
        try:
            price_x_kg = price_x_kg.string
            price_x_kg = price_x_kg.split("$")[1].strip(")").strip(" x kg")
            info.append(price_x_kg)
        except:
            info.append(None)
    else:
        info.append(None)
    return info
    

def get_info_from_url(url, retries=0):
    if (retries > 5):
      return []
    # try:
    html_soup = get_html_soup(url)
    info = []
    info = get_codes(html_soup, info, tags=['div', 'span', 'h1', 'h2', 'p'], classes=["product-code"])
    info = get_names(html_soup, info, tags=['div', 'span', 'h1', 'h2', 'p'], classes=["product-name"])
    info = get_photo(html_soup, info, tags=['div', 'span', 'h1', 'h2', 'p'], classes=["zoomed-image"])
    info = get_price(html_soup, info, tags=['div', 'span', 'h1', 'h2', 'p'], classes=["price-best", "product-sigle-price-wrapper"])
    info = get_price_kg(html_soup, info, tags=['div', 'span', 'h1', 'h2', 'p'], classes=["product-price-ppum"])
    return info
    # except:
    #   print("Something went wrong, retrying")
    #   get_info_from_url(url, retries= retries + 1)


def search_tomates_jumbo(tomates):
    lista_columnas = ["Code", "Name", "Photo", "Price", "Price x kg"]
    tomates_filtrados = []
    for index in tqdm(range(len(tomates))):
        info = get_info_from_url(tomates[index])
        tomates_filtrados.append(info)
    print(tomates_filtrados)
    df = pd.DataFrame(tomates_filtrados, columns=lista_columnas)
    return df

ModuleNotFoundError: No module named 'bs4'

In [ ]:
tomates_jumbo = [
    "https://www.jumbo.cl/tomate-larga-vida-granel/p",
    "https://www.jumbo.cl/tomate-malla-1-kg/p",
    "https://www.jumbo.cl/tomate-salad-granel/p",
    "https://www.jumbo.cl/tomate-jumbo-granel-beef-2/p"
]

df = search_tomates_jumbo(tomates_jumbo)
